In [1]:
from langchain.prompts.prompt import PromptTemplate
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain import LLMChain
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage, OutputParserException
from langchain.callbacks.base import BaseCallbackManager
from langchain_experimental.sql import SQLDatabaseChain
from langchain import OpenAI, SQLDatabase
from langchain.chains import SimpleSequentialChain
from langchain.memory import ConversationBufferWindowMemory

import langchain
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate, ChatPromptTemplate
from langchain import LLMChain
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage, OutputParserException
from langchain.callbacks.base import BaseCallbackManager
from langchain_experimental.sql import SQLDatabaseChain
import re
import os

/Users/apple/Documents/hackathon_ai/ai-commerce-insights/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=10000)
username = 'root'
password = 'mindstix'
host = 'localhost'  # or your database host
port = '3306'       # default MySQL port
database = 'edb2'
table_name = 'amazon_sale_report'
sql_string = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}'

# db = SQLDatabase.from_uri("sqlite:///Chinook.db")
db = SQLDatabase.from_uri(sql_string)
db.get_table_names()

/var/folders/9r/89zd4rts6k5gy4qdyjc9nsqw0000gn/T/ipykernel_56423/3139693646.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=10000)
/var/folders/9r/89zd4rts6k5gy4qdyjc9nsqw0000gn/T/ipykernel_56423/3139693646.py:12: LangChainDeprecationWarning: The method `SQLDatabase.get_table_names` was deprecated in langchain-community 0.0.1 and will be removed in 1.0. Use get_usable_table_names instead.
  db.get_table_names()


['amazon_sale_report']

In [3]:
memory = ConversationBufferWindowMemory(k=2,memory_key="chat_history", return_messages=True)

# _DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
# Use the following format:

# Question: "Question here"
# SQLQuery: "SQL Query to run"
# SQLResult: "Result of the SQLQuery"
# Answer: "Final answer here"

# Only use the following tables:

# {table_info}


# If some one asks for details of a product name,always create SQLQuery, with LIKE query.
# Give the answer as humanly as possible.
# Question: {input}"""

# PROMPT = PromptTemplate(
#     input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
# )
# db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=False)



In [4]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

# Load other tools
from langchain.agents import load_tools
tools = load_tools(
    [],
    llm=llm
)

# Create toolkit and get tools
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools2 = toolkit.get_tools()[:1]

# Extend the tools list with the tools2 list
tools.extend(tools2)
tools


[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10d7d0ca0>)]

In [5]:
# from langchain_community.agent_toolkits import create_sql_agent

# db_chain = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

# database_tool = Tool(
#         name='MYSQL_DB',
#         func=db_chain.run,
#         description=(
#             'use this tool for answering questions related to data need from mysql db'
#         )
#     )


# from langchain.agents import load_tools
# tools = load_tools(
#     [],#"llm-math"],
#     llm=llm
#     # input_func=get_input
# )
# tools.append(database_tool)


In [6]:


template_with_history = """
If the user is greeting, no action is required. Answer everything in a personalized way using the available user data, and include the user's first name in the answer.

You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer  
Thought: you should always think about what to do  
Action: the action to take, should be one of [{tool_names}] names  
Action Input: the question or SQL query to the action, should be human-like, clear, and relevant to the database  
Observation: the result of the action  
... (this Thought/Action/Action Input/Observation can repeat N times)  
Thought: I now know the final answer  
Final Answer: the final answer to the original input question

Use the following instructions:

For the Action Input to MYSQL_DB:
    1. Please formulate a **SQL query** specifically based on the attributes of this tool and the question being asked. Ensure the query is correct, considering the schema, table names, and columns.
    2. Always create SQLQuery, with LIKE query.
    3. The schema provided is:
    
    Table: 'amazon_sale_report'  
    Description: This dataset provides detailed insights into Amazon sales data, including SKU Code, Design Number, Stock, Category, Size, and Color, to help optimize product profitability.  
    Schema:  
        CREATE TABLE amazon_sale_report (  
        `index` INTEGER,  
        order_id TEXT,  
        date DATE,  
        status TEXT,  
        fulfilment TEXT,  
        sales_channel TEXT,  
        `ship-service-level` TEXT,  
        style TEXT,  
        `SKU` TEXT,  
        category TEXT,  
        size TEXT,  
        `ASIN` TEXT,  
        courier_status TEXT,  
        qty INTEGER,  
        currency TEXT,  
        amount DOUBLE,  
        `ship-city` TEXT,  
        `ship-state` TEXT,  
        `ship-postal-code` INTEGER,  
        `ship-country` TEXT,  
        `promotion-ids` TEXT,  
        `B2B` TEXT,  
        `fulfilled-by` TEXT,  
        unnamed TEXT  
        )DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

    3. Tailor your SQL query to fetch the most relevant data to answer the user's question. Ensure clarity in your query to get accurate results.

For the Final Answer:
    1. User is not aware of the internal observations or steps, so generate the output with clarity, and summarize based on the query result.
    2. Tone: friendly + professional, aiming to provide clear and insightful information.
    3. If the answer involves numerical data, make sure to present it accurately and concisely (e.g., round off floating numbers if applicable).

Instructions: Maintain a friendly and professional tone throughout the conversation. Provide personalized answers and recommendations where possible.

Begin!
{chat_history}

Question: {input}
{agent_scratchpad}
"""

In [7]:
from langchain_core.tools import BaseTool, Tool

class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[BaseTool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]
    


In [8]:

prompt = CustomPromptTemplate(
    template=template_with_history,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps","chat_history"] 
)
FINAL_ANSWER_ACTION = "Final Answer:"
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        includes_answer = FINAL_ANSWER_ACTION in text
        regex = (
            r"Action\s*\d*\s*:[\s]*(.*?)[\s]*Action\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        )
        action_match = re.search(regex, text, re.DOTALL)
        if action_match:
            if includes_answer:
                raise OutputParserException(
                    "Parsing LLM output produced both a final answer "
                    f"and a parse-able action: {text}"
                )
            action = action_match.group(1).strip()
            action_input = action_match.group(2)
            tool_input = action_input.strip(" ")
            # ensure if its a well formed SQL query we don't remove any trailing " chars
            if tool_input.startswith("SELECT ") is False:
                tool_input = tool_input.strip('"')

            return AgentAction(action, tool_input, text)

        elif includes_answer:
            return AgentFinish(
                {"output": text.split(FINAL_ANSWER_ACTION)[-1].strip()}, text
            )

        if not re.search(r"Action\s*\d*\s*:[\s]*(.*?)", text, re.DOTALL):
            raise OutputParserException(
                f"Could not parse LLM output: `{text}`",
                observation="Invalid Format: Missing 'Action:/Final Answer:' after 'Thought:'",
                llm_output=text,
                send_to_llm=True,
            )
        elif not re.search(
            r"[\s]*Action\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)", text, re.DOTALL
        ):
            raise OutputParserException(
                f"Could not parse LLM output: `{text}`",
                observation="Invalid Format:"
                " Missing 'Action Input:' after 'Action:'",
                llm_output=text,
                send_to_llm=True,
            )
        else:
            raise OutputParserException(f"Could not parse LLM output: `{text}`")

from langchain.agents.mrkl.output_parser import MRKLOutputParser   
# from langchain.agents.conversational_chat.output_parser import ConvoOutputParser
output_parser = MRKLOutputParser()                            #MRKLOutputParser()

callback_manager = BaseCallbackManager

llm_chain = LLMChain(llm=llm, prompt=prompt)

tool_names = [tool.name for tool in tools]
print(tool_names)
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names,
    callback_manager = callback_manager,
    # handle_parsing_errors = True
    # memory = memory
)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory = memory, handle_parsing_errors=True, )
# agent_executor.max_iterations(10)


['sql_db_query']


/var/folders/9r/89zd4rts6k5gy4qdyjc9nsqw0000gn/T/ipykernel_56423/644728112.py:63: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
/var/folders/9r/89zd4rts6k5gy4qdyjc9nsqw0000gn/T/ipykernel_56423/644728112.py:67: LangChainDeprecationWarning: Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.
  agent = LLMSingleActionAgent(


In [9]:
user_question = 'the sales are down in june 2022 as compared to may 2022, what may be the possible reasons behind this. can you provide few datapoints and numeric results of analysis based on the data available to back your reasons'

agent_executor.run(user_question)


/var/folders/9r/89zd4rts6k5gy4qdyjc9nsqw0000gn/T/ipykernel_56423/770682629.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  agent_executor.run(user_question)




> Entering new AgentExecutor chain...
Thought: To analyze the sales data for June 2022 compared to May 2022, I need to query the sales figures for both months. This will help identify any trends or significant changes in sales that could explain the decline. 

Action: sql_db_query  
Action Input: SELECT date, SUM(amount) AS total_sales FROM amazon_sale_report WHERE date BETWEEN '2022-05-01' AND '2022-05-31' GROUP BY date UNION ALL SELECT date, SUM(amount) AS total_sales FROM amazon_sale_report WHERE date BETWEEN '2022-06-01' AND '2022-06-30' GROUP BY date;  

Observation:[(datetime.date(2022, 5, 31), 903069.0900000002), (datetime.date(2022, 5, 30), 817463.7699999999), (datetime.date(2022, 5, 29), 816996.4900000001), (datetime.date(2022, 5, 28), 850986.6000000002), (datetime.date(2022, 5, 27), 813334.57), (datetime.date(2022, 5, 26), 767619.5999999999), (datetime.date(2022, 5, 25), 819478.3400000001), (datetime.date(2022, 5, 24), 759458.39), (datetime.date(2022, 5, 23), 755930.56), (d

KeyboardInterrupt: 

In [ ]:
db_chain.run("give nuber of order with amount less than 1000")

In [ ]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain.chains import create_sql_query_chain
from langchain_experimental.sql import SQLDatabaseChain

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
# chain = write_query | execute_query
write_query.invoke({"question": "give nuber of order with amount less than 1000"})

In [11]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=db_chain).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

# chain.invoke({"question": "How many employees are there"})

In [ ]:
chain.invoke({"question": "give nuber of order with amount less than 1000"})

In [ ]:
db_chain.run("give nuber of order with amount less than 1000")

In [ ]:

database_tool = Tool(
        name='Shop DB',
        func=overall_chain.run,
        description=(
            'use this tool for answering questions related to SHOP details of products like SHOP NAME, SHOP LOCATION, SHOP LANDMARK, SHOP CITY, SHOP STATE, PINCODE.' \
            'And if they are AVAILABLE in the INVENTORY or NOT, and their COUNT.'
        )
    )


tools = load_tools(
    [],#"llm-math"],
    llm=llm
    # input_func=get_input
)

tools.append(knowledge_base_tool)
tools.append(database_tool)
 